[데이터사이언스스쿨](https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/)
  
[텐서플로와 머신러닝으로 시작하는 자연어 처리](https://book.naver.com/bookdb/book_detail.nhn?bid=14488487)

# 단어를 표현하는 방법

## 개요

### 원 핫 인코딩
> 각 단어의 인덱스를 정한 후 각 단어의 벡터를 그 단어에 해당하는 인덱스의 값을 1로 표현하는 방법
- 문제점
    - 1. 희소문제: 단어 벡터의 크기가 너무 커지면 공간을 많이 사용한다.(공간에 비해 실 사용 값은 1 하나 뿐이므로 비효율적)
    - 2. 값 자체는 단순히 단어가 무엇인지만 알려주고, 의미나 특성 등이 전혀 표현되지 않는다.
    
위와 같은 문제들을 해결하기 위해서 분포 가설(Distributed hypothesis)을 기반으로 한 방법들이 제안되었다.
> 분포가설이란, '같은 문맥의 단어. 즉, 비슷한 위치에 나오는 단어는 비슷한 의미를 가진다'는 의미

### 카운트 기반 방법
> 어떤 글의 문맥 안에 단어가 동시에 등장하는 횟수를 세는 방법
- 동시 등장 횟수를 행렬로 나타낸 뒤 그 행렬을 수치화해서 단어 벡터로 만든다.

- 장점
    - 빠르다. 단어 벡터가 많아져도 시간이 크게 증가하지 않는다.
    - 데이터가 많을 경우, 단어가 잘 표현되어 효율적이다.

### 예측 방법
> 신경망 구조 혹은 모델을 사용해서 특정 문맥에서 단어가 나올지를 예측하면서 단어를 벡터로 만드는 방식  


[워드 임베딩](https://github.com/InkyunMoon/TIL/blob/master/4.%20NLP/Word%20Embedding.ipynb)
- 장점
    - 카운트 기반 방법보다 단어 간의 유사도를 잘 측정한다.
    - 단어들의 복잡한 특징까지도 잘 잡아낸다.
    - 만들어진 단어 벡터는 서로에게 유의미한 관계를 측정할 수 있다.
        - 엄마-아빠 사이의 거리와 여자-남자 사이의 거리가 비슷하게 나온다.
        
        
보통의 경우 예측 기반 방법의 성능이 좋아서 예측 기반 방법을 사용하며, 두 방법을 모두 포함하는 Glove라는 방법도 있다.
그러나, 항상 좋은 성능을 내는 방법이 있는 것은 아니므로 각 방법간 차이점을 숙지하고 상황에 맞게 사용해야한다.

## 1. Hashing trick을 이용한 word embedding
- Vocabulary의 크기를 미리 지정하고 hash함수 혹은 md5함수를 이용하여 단어들을 hash table에 대응시키는 방식
    - hash table : 넓은 범위의 키 값을 작은 메모리 영역에 대응시키는 방법. 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import hashing_trick
from tensorflow.keras.layers import Input, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.

In [2]:
samples = ['너 오늘 이뻐 보인다.',
          '나는 오늘 기분이 더러워',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']
labels = [[1],[0],[1],[1],[0],[1]]

In [3]:
samples = ['너 오늘 이뻐 보인다.',
          '나는 오늘 기분이 더러워',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']
labels = [[1],[0],[1],[1],[0],[1]]

In [4]:
VOCAB_SIZE = 10
sequences = [hashing_trick(s, VOCAB_SIZE) for s in samples]
sequences = np.array(sequences)
labels = np.array(labels)
print(sequences)

[[9 4 1 6]
 [6 4 8 1]
 [4 8 5 1]
 [6 4 6 3]
 [9 4 7 1]]


In [5]:
# Embedding layer 내부 출력층의 개수
EMB_SIZE = 8

xInput = Input(batch_shape=(None,sequences.shape[1]))
embed_input = Embedding(input_dim = VOCAB_SIZE + 1, output_dim = EMB_SIZE)(xInput)
embed_input1 = tf.reduce_mean(embed_input, axis=-1)

In [6]:
hidden_layer = Dense(128, activation = tf.nn.relu)(embed_input1)
output = Dense(1, activation = 'sigmoid')(hidden_layer)
model = Model(xInput, output)
model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr=0.01))

In [7]:
model.fit(sequences, labels, epochs=100)
pred = model.predict(sequences)
print(np.round(pred,0))

ValueError: Data cardinality is ambiguous:
  x sizes: 5
  y sizes: 6
Please provide data which shares the same first dimension.

# 2. CountVectorizer & Co-occurence matrix

### CountVectorizer
> 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW인코딩한 벡터를 만든다.

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?'
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'this': 9,
 'is': 3,
 'the': 7,
 'first': 2,
 'document': 1,
 'second': 6,
 'and': 0,
 'third': 8,
 'one': 5,
 'last': 4}

In [33]:
vect.transform(['This is the second second documner.']).toarray()

array([[0, 0, 0, 1, 0, 0, 2, 1, 0, 1]], dtype=int64)

In [34]:
vect.transform(corpus).toarray()

array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]], dtype=int64)

CountVectorizer의 인수들
- stop_words
vect = CountVectorizer(stop_words=['and','is','the','this']).fit(corpus)
- analyzer
- token_pattern
- tokenizer
- ngram_range
- max_df
- min_df

In [35]:
# stop_words
vect = CountVectorizer(stop_words=['and','is','the','this']).fit(corpus)
vect.vocabulary_

{'first': 1, 'document': 0, 'second': 4, 'third': 5, 'one': 3, 'last': 2}

In [27]:
vect = CountVectorizer().fit_transform(corpus)
print(vect)

  (0, 9)	1
  (0, 3)	1
  (0, 7)	1
  (0, 2)	1
  (0, 1)	1
  (1, 9)	1
  (1, 3)	1
  (1, 7)	1
  (1, 1)	1
  (1, 6)	2
  (2, 7)	1
  (2, 0)	1
  (2, 8)	1
  (2, 5)	1
  (3, 9)	1
  (3, 3)	1
  (3, 7)	1
  (3, 2)	1
  (3, 1)	1
  (4, 7)	1
  (4, 1)	1
  (4, 4)	1


### CountVectorizer 2

In [48]:
docs = ['성진과 창욱은 야구장에 갔다',
       '성진과 태균은 도서관에 갔다',
       '성진과 창욱은 공부를 좋아한다']

count_model = CountVectorizer(ngram_range=(1,1))
x = count_model.fit_transform(docs)
vocab_size = count_model.vocabulary_
print(vocab_size)

{'성진과': 3, '창욱은': 6, '야구장에': 4, '갔다': 0, '태균은': 7, '도서관에': 2, '공부를': 1, '좋아한다': 5}


In [38]:
# Compact Sparse Row(CSR) format
print(x)

  (0, 3)	1
  (0, 6)	1
  (0, 4)	1
  (0, 0)	1
  (1, 3)	1
  (1, 0)	1
  (1, 7)	1
  (1, 2)	1
  (2, 3)	1
  (2, 6)	1
  (2, 1)	1
  (2, 5)	1


In [39]:
print(x.toarray()) # DTM형태
print()
print(x.T.toarray())

[[1 0 0 1 1 0 1 0]
 [1 0 1 1 0 0 0 1]
 [0 1 0 1 0 1 1 0]]

[[1 1 0]
 [0 0 1]
 [0 1 0]
 [1 1 1]
 [1 0 0]
 [0 0 1]
 [1 0 1]
 [0 1 0]]


In [46]:
# Co-occurence matrix를 만든다.
xc = x.T * x
xc.setdiag(0)

In [47]:
print(xc.toarray())

[[0 0 1 2 1 0 1 1]
 [0 0 0 1 0 1 1 0]
 [1 0 0 1 0 0 0 1]
 [2 1 1 0 1 1 2 1]
 [1 0 0 1 0 0 1 0]
 [0 1 0 1 0 0 1 0]
 [1 1 0 2 1 1 0 0]
 [1 0 1 1 0 0 0 0]]


### CountVectorizer의 ngram_range인자를 (1,2)로 설정해보자.

In [50]:
count_model = CountVectorizer(ngram_range=(1,2))
x = count_model.fit_transform(docs)

print(count_model.vocabulary_)

{'성진과': 5, '창욱은': 11, '야구장에': 8, '갔다': 0, '성진과 창욱은': 6, '창욱은 야구장에': 13, '야구장에 갔다': 9, '태균은': 14, '도서관에': 3, '성진과 태균은': 7, '태균은 도서관에': 15, '도서관에 갔다': 4, '공부를': 1, '좋아한다': 10, '창욱은 공부를': 12, '공부를 좋아한다': 2}


In [52]:
xc = x.T * x
xc.setdiag(0)
print(xc.toarray())

[[0 0 0 1 1 2 1 1 1 1 0 1 0 1 1 1]
 [0 0 1 0 0 1 1 0 0 0 1 1 1 0 0 0]
 [0 1 0 0 0 1 1 0 0 0 1 1 1 0 0 0]
 [1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1]
 [1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 1]
 [2 1 1 1 1 0 2 1 1 1 1 2 1 1 1 1]
 [1 1 1 0 0 2 0 0 1 1 1 2 1 1 0 0]
 [1 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1]
 [1 0 0 0 0 1 1 0 0 1 0 1 0 1 0 0]
 [1 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0]
 [0 1 1 0 0 1 1 0 0 0 0 1 1 0 0 0]
 [1 1 1 0 0 2 2 0 1 1 1 0 1 1 0 0]
 [0 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0]
 [1 0 0 0 0 1 1 0 1 1 0 1 0 0 0 0]
 [1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1]
 [1 0 0 1 1 1 0 1 0 0 0 0 0 0 1 0]]


### 위에서 구한 Co-occurence matrix를 SVD로 분해해보자.

In [57]:
C = xc.toarray()
U, S ,VT = np.linalg.svd(C, full_matrices= True)
print(np.round(U,2), '\n') # m*m
print(np.round(S,2), '\n') # m*n
print(np.round(VT,2), '\n') # n*m

[[-0.33  0.25 -0.2   0.28 -0.   -0.79  0.3   0.   -0.   -0.    0.   -0.
  -0.    0.   -0.   -0.  ]
 [-0.19 -0.22  0.32  0.07  0.   -0.13 -0.21 -0.07 -0.5  -0.22  0.49 -0.03
  -0.17  0.27 -0.33  0.01]
 [-0.19 -0.22  0.32  0.07  0.   -0.13 -0.21  0.08  0.46 -0.17 -0.11  0.1
   0.02 -0.02 -0.01  0.69]
 [-0.15  0.35  0.13  0.09 -0.    0.14 -0.09  0.12 -0.51  0.17 -0.56 -0.03
  -0.2  -0.16 -0.21  0.27]
 [-0.15  0.35  0.13  0.09  0.    0.14 -0.09  0.16 -0.15  0.32  0.52  0.08
   0.21 -0.19  0.49  0.23]
 [-0.47  0.06  0.04 -0.87  0.   -0.04  0.16 -0.    0.   -0.   -0.    0.
  -0.    0.   -0.    0.  ]
 [-0.37 -0.24 -0.06  0.24  0.71  0.32  0.38  0.   -0.    0.   -0.   -0.
  -0.   -0.   -0.   -0.  ]
 [-0.15  0.35  0.13  0.09  0.    0.14 -0.09 -0.33  0.17 -0.54  0.09  0.11
  -0.16 -0.52 -0.05 -0.21]
 [-0.21 -0.06 -0.39  0.02 -0.    0.05 -0.36  0.58 -0.11 -0.44 -0.06 -0.15
   0.29 -0.05  0.06 -0.1 ]
 [-0.21 -0.06 -0.39  0.02  0.    0.05 -0.36 -0.44  0.04  0.12 -0.   -0.49
  -0.34  0.09  0.27  0.1

In [59]:
# S가 대각선 성분만 리턴되었으므로 정방행렬로 바꾼다.

s = np.diag(S)
print(np.round(s,2))

[[9.24 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   4.88 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   2.74 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   2.65 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   1.81 0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   1.4  0.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.
  0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.
  0.   0.  ]
 [0.

### SVD의 차원을 축소한다.

In [80]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components = 4, n_iter = 7)
D = svd.fit_transform(xc.toarray())

In [81]:
svd.singular_values_

array([9.24239861, 4.88199345, 2.73513827, 2.65397905])

In [82]:
U = D / svd.singular_values_
S = np.diag(svd.singular_values_)
VT = svd.components_

In [91]:
# C를 4개의 차원으로 축소
U@S

array([[ 3.06865075,  1.23647381,  0.56000039, -0.74876096],
       [ 1.78329007, -1.07282969, -0.86432257, -0.18455785],
       [ 1.78329007, -1.07282969, -0.86432253, -0.18455804],
       [ 1.40748621,  1.71823003, -0.36323929, -0.23273073],
       [ 1.40748621,  1.71823003, -0.36323939, -0.23273021],
       [ 4.309953  ,  0.27899382, -0.10055292,  2.29734383],
       [ 3.41102723, -1.14902613,  0.16473944, -0.6269283 ],
       [ 1.40748621,  1.71823003, -0.36323925, -0.23273041],
       [ 1.96076728, -0.27154282,  1.0731673 , -0.06332774],
       [ 1.96076728, -0.27154282,  1.07316733, -0.06332764],
       [ 1.78329007, -1.07282969, -0.86432253, -0.18455797],
       [ 3.41102723, -1.14902613,  0.16473944, -0.6269283 ],
       [ 1.78329007, -1.07282969, -0.86432254, -0.18455825],
       [ 1.96076728, -0.27154282,  1.07316726, -0.06332803],
       [ 1.40748621,  1.71823003, -0.36323928, -0.23273017],
       [ 1.40748621,  1.71823003, -0.36323922, -0.23273022]])